# Laboratorio 7 - Parte 1

## Integrantes

- Sergio Orellana 221122
- Rodrigo Mansilla 22611
- Carlos Valladares 221164

## Parte 1

### Pregunta 1
**Describa con sus propias palabras la diferencia fundamental entre una Entidad de DES y un Agente de MBA. ¿Por qué esta diferencia hace que cada paradigma sea adecuado para tipos de problemas distintos? Proporcione un ejemplo de un problema de modelado para cada uno**

En DES, una *entidad* (por ejemplo, un cliente o una orden) es típicamente pasiva: fluye a través de un proceso predefinido (llegar → esperar → ser atendida → salir) y “cobra vida” solo cuando ocurre un evento del sistema (arriba una llegada, termina un servicio, cambia un recurso). La lógica dominante está en la línea de proceso y los recursos, no dentro de la entidad.  
En MBA, un *agente* (por ejemplo, una persona, empresa o vehículo) es autónomo: tiene estado interno, reglas de decisión, objetivos e interacciones con otros agentes y con su entorno; actúa continuamente en el tiempo o al ritmo de sus propias reglas, generando comportamientos emergentes.

Esta diferencia hace que DES sea ideal para sistemas de flujo y capacidad (colas, procesos operativos) donde el interés es medir tiempos de espera, utilización de recursos y throughput. En cambio, MBA es adecuado para sistemas complejos con heterogeneidad y retroalimentaciones micro-macro (movilidad, propagación, mercados, redes) donde importan interacciones locales y decisiones individuales.

- **Ejemplo DES:** un centro de llamadas con llegadas de clientes, múltiples agentes telefónicos, prioridades, *SLA* y reintentos.  
- **Ejemplo MBA:** propagación de rumores o enfermedad en una ciudad con individuos que se mueven, deciden y cambian su comportamiento según su red social y percepciones de riesgo.

### Pregunta 2
**Explique por qué las colas son una consecuencia inevitable en la mayoría de los sistemas modelados con DES. ¿Qué tipo de información valiosa podemos extraer al analizar la longitud de una cola y el tiempo de espera en ella a lo largo de una simulación?**

Con llegadas y tiempos de servicio variables (incertidumbre), incluso si la utilización promedio es menor que 1, la variabilidad provoca congestión temporal: llegan “ráfagas” y los servicios tienen duraciones irregulares, así que se forman colas. En régimen estable, la teoría de colas muestra que la relación entre el número promedio en el sistema $L$, la tasa de llegada efectiva $\lambda$ y el tiempo promedio en el sistema $W$ cumple la Ley de Little: $L=\lambda W$. Cuanto mayor es la utilización y la variabilidad (de llegadas y servicio), mayor será la cola y el tiempo de espera esperados.

Al analizar longitud de cola y tiempos de espera (promedios, percentiles y colas de distribución) puedo:
- Detectar cuellos de botella y puntos de saturación.  
- Dimensionar recursos (servidores, puestos, inventario) para cumplir SLA.  
- Evaluar políticas (prioridades, citas, *pooling* de recursos, *batching*).  
- Cuantificar riesgo operativo (picos, *tail risk*).  
- Proponer trade-offs costo–servicio (por ejemplo, cuántos servidores adicionales reducen el $95^\text{o}$ percentil de espera a un objetivo).

### Pregunta 3
**En clase se mencionaron los recursos móviles, estáticos y portátiles. Describa un escenario (diferente al hospital) que involucre los tres tipos de recursos. Identifique qué elementos serían cada tipo de recurso y cómo interactuarían.**

Centro de distribución de e‑commerce:
- **Recursos estáticos:** estaciones de picking y empaque fijas, y cintas transportadoras en ubicaciones específicas.  
- **Recursos móviles:** robots AMR o montacargas que se desplazan entre racks, estaciones y muelles. Son solicitados para transportar pedidos o contenedores.  
- **Recursos portátiles:** totes (contenedores) y escáneres de mano que acompañan al pedido mientras avanza por el proceso; se “apoderan” (seize) al inicio y se devuelven (release) al terminar.

**Interacción:** una orden llega, se apodera de un tote (portátil) y solicita un robot (móvil) que la lleva a la estación de picking (estática). Tras el picking, otro traslado móvil la lleva a empaque (estático); el tote permanece con la orden hasta el despacho y luego retorna al pool de portátiles. 

### Pregunta 4
**Imagine que está modelando la propagación de una enfermedad en una ciudad (un problema clásico de MBA). Ahora, quieres añadir el proceso de vacunación en centros de salud. ¿Cómo podría combinar MBA y DES? ¿Qué parte del modelo sería MBA y cuál DES? ¿Qué información pasaría de un modelo a otro?**

Construiría un modelo híbrido:
- **Parte MBA (ciudad):** agentes personas con atributos (edad, estado $S/E/I/R/V$, aceptación vacunal), movilidad diaria y red de contactos. La transmisión ocurre por interacciones y decisiones individuales (por ejemplo, acudir o no a vacunarse).  
- **Parte DES (centros de salud):** flujo de pacientes por recepción → vacunación → observación, con recursos (personal, cabinas, dosis disponibles), colas, tiempos de servicio y horarios. Se mide throughput, esperas y utilización.

Intercambio de información:  
- **MBA a DES:** listas (o tasas) de llegadas por centro y franja horaria (derivadas de decisiones/ubicación de agentes), no‑shows, prioridad por riesgo.  
- **DES a MBA:** eventos de vacunación confirmados por individuo y tiempo (cambian el estado del agente a $V$ tras un delay de efectividad), tiempos de espera (que pueden afectar comportamiento futuro), y capacidad/stock (que condiciona citas futuras).  
El acople puede ser sincrónico (timestep común) o por eventos (el DES dispara eventos de vacunación que el MBA consume). Así preservo la dinámica micro de contagios (MBA) y el rendimiento operativo de la vacunación (DES).

## Práctica


#### Inicialización del entorno y parámetros

In [ ]:
import simpy
import random
import statistics

NUM_RECEPCIONISTAS = 1
NUM_MEDICOS = 2
TIEMPO_REGISTRO_PROMEDIO = 2.0      
TIEMPO_CONSULTA_PROMEDIO = 7.0      
TASA_LLEGADA_PACIENTES = 5.0        
TIEMPO_SIMULACION = 120.0          

# Lista para almacenar los tiempos totales de permanencia
tiempos_de_espera_totales = []


**Referencias:** 

* Mustafee, N., & Powell, J. H. (2018). *From hybrid simulation to hybrid systems modelling.* Procedings of the Winter Simulation Conference. Disponible en: [https://ore.exeter.ac.uk/articles/conference_contribution/From_Hybrid_Simulation_to_Hybrid_Systems_Modelling/29749883](https://ore.exeter.ac.uk/articles/conference_contribution/From_Hybrid_Simulation_to_Hybrid_Systems_Modelling/29749883) ([ore.exeter.ac.uk][1])
* North, M. J., Collier, N. T., & Vos, J. R. (2006). *Experiences creating three implementations of the Repast Agent Modeling Toolkit.* ACM Transactions on Modeling and Computer Simulation. Disponible en: [https://dl.acm.org/doi/10.1145/1122012.1122013](https://dl.acm.org/doi/10.1145/1122012.1122013) ([dl.acm.org][2])
* Helbing, D. (2001). *Traffic and related self-driven many-particle systems.* Reviews of Modern Physics, 73(4), 1067-1141. Disponible en: [https://link.aps.org/doi/10.1103/RevModPhys.73.1067](https://link.aps.org/doi/10.1103/RevModPhys.73.1067) ([link.aps.org][3])
* Epstein, J. M. (2009). *Modelling to contain pandemics.* Nature, 460(7256), 687-687. Disponible en: [https://www.nature.com/articles/460687a](https://www.nature.com/articles/460687a) ([nature.com][4])